In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/user/project/sharp_features/')

In [2]:
from sharpf.models import load_model, MODEL_BY_NAME
from sharpf.modules.base import load_with_spec
from sharpf.modules import module_by_kind
import torch
import numpy as np

import json
spec = json.load(open('/home/user/project/sharp_features/sharpf/models/specs/dgcnn.json', 'r'))

from sharpf.data.data import ABCData

data_label = 'points'
target_label = 'distances'
data_root = '/home/user/project/point_patches/'
dtst = ABCData(
    data_path=data_root, partition='train',
    data_label=data_label,
    target_label=target_label
)

12208593.0 out of 4733 items are sharp


In [3]:
from torch import nn

def weights_init(m):
    seed = 139
    torch.cuda.manual_seed(seed)
    modules = list(m.children())
    if not modules:
        if isinstance(m, nn.Conv2d):
            weight = cached_weights.get(str(m) + '_weight', None)
            bias = cached_weights.get(str(m) + '_bias', None)
            if weight is not None:
                m.weight = torch.nn.Parameter(weight)
                if bias is not None:
                    m.bias = torch.nn.Parameter(bias)
            else:
                nn.init.xavier_uniform_(m.weight)
                cached_weights[str(m) + '_weight'] = m.weight.detach()
                if m.bias is not None:
                    nn.init.xavier_uniform_(m.bias)
                    cached_weights[str(m) + '_bias'] = m.bias.detach()
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)
        return
    else:
        for item in modules:
            weights_init(item)

In [9]:
from model_seg import DGCNN
from collections import namedtuple
torch.cuda.empty_cache()


device = 'cuda:2'
points = dtst[0][0]
points = torch.tensor(np.stack([points for idx in range(10)]), device=device)

artons_model = load_with_spec(spec, MODEL_BY_NAME).to(device).eval()
Args = namedtuple('Args', ['k', 'dropout'])
args = Args(10, 0.5)
dgcnn_model = DGCNN(args).to(device).eval()

N = 100
res_eq = []
for idx in range(N):
    cached_weights = {}
    weights_init(dgcnn_model)
    weights_init(artons_model)

    art_res = artons_model(points)
    dgcnn_res = dgcnn_model(points.permute(0,2,1))
    res_eq.append(torch.equal(art_res, dgcnn_res))
    if not torch.equal(art_res, dgcnn_res):
        print('-----------')
        print(art_res)
        print(dgcnn_res)
        print('-----------')
print(torch.tensor(res_eq).all())

-----------
tensor([[0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        ...,
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952]],
       device='cuda:2', grad_fn=<SqueezeBackward0>)
tensor([[0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        ...,
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952],
        [0.4394, 0.6323, 0.5088,  ..., 0.5647, 0.5794, 0.4952]],
       device='cuda:2', grad_fn=<SqueezeBackward0>)
-----------
-----------
tensor([[0.6224, 0.5878, 0.6186,  ..., 0.6279, 0.6783, 0.6340],


In [7]:
res_eq

[False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [38]:
dgcnn_model

DGCNN(
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn7): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn8): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2

In [5]:
artons_model.decoder_blocks[2]._op[2].conv[0]

Sequential(
  (0): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): ConvBase()
  (2): ConvBase()
)

In [31]:
dgcnn_model.conv10

Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)